In [2]:
# ## dependencies for colab
# !git clone https://__TOKEN_GIT__:@github.com/DanielSc4/RewardLM.git
# %cd RewardLM/
# !pip install -r requirements.txt
# from huggingface_hub import login
# login(token = '')  # https://huggingface.co/settings/tokens

import os
def init():
    # disable welcome message
    os.environ['BITSANDBYTES_NOWELCOME'] = '1'
    # os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

def update_code():
    !git pull
    %cd RewardLM/

init()

# 🥞 Reinforcement Learning with Automatic Feedback (RLAF)

Generation config from [here](https://github.com/LAION-AI/Open-Assistant/blob/main/model/model_training/configs/ppo_config.yaml) seems to work

In [ ]:
# test
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('AlekseyKorshuk/vicuna-7b')

In [1]:
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
data = get_DIALOCONAN_prepro(delete_last_assistant_response = True)
len(data)

8312

In [ ]:
import torch
from rewardlm.core.RL.RLModel import RLModel
from rewardlm.data.data_utils import get_DIALOCONAN_prepro
from transformers import GenerationConfig
from rewardlm.utils import load_config

config = load_config('debug_RL')

rlmanager = RLModel(
    model_id = config['generation']['model_id'],
    reward_model_id = config['reward']['model_id'],
    optimized = True,   # use LoRA
    bs = config['PPO']['bs'],
    mini_bs = config['PPO']['mini_bs'],
    # force the use of CPU on Apple Silicon devices (mps not supported):
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
    generation_config=GenerationConfig(
        max_new_tokens = 512,
        min_new_tokens = 5,
        pad_token_id = 0,       # crashes while using batchsize > 1 only on mps device if not set
        temperature = 1,
        top_p = .7,
        top_k = 0,
        do_sample = True
        # diversity_penalty = .1, # should use num_beam_groups > 1
    )
)

data = get_DIALOCONAN_prepro(delete_last_assistant_response = True)
if config['data']['subset']:
    # select only the first `subset_size` samples
    data = data[:config['data']['subset_size']]
dataset = rlmanager.generate_dataset(text = data)

stats = rlmanager.train_PPO(dataset = dataset)
print('Done')

# assuming debug if subset is active
if not config['data']['subset']:
    # save trainer (model, tokenizer & config) to the hub
    repo_id = 'DanielSc4/' + config['generation']['model_id'].split('/')[1] + '-RL-LoRA-test0'

    rlmanager.push_generator_to_hub(repo_id = repo_id)

: 

# 👨🏼‍🏫 Model fine-tune

In [1]:
import torch
from transformers import GenerationConfig
from rewardlm.data.data_utils import get_DIALOCONAN_prepro, get_dataset_CLM
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.utils import load_config

config = load_config('debug_notGPT_FT')
repo_id = 'DanielSc4/' + config['generation']['model_id'].split('/')[1] + '-FT-LoRA-test1'

generator_manager = GenerativeModel(
    config['generation']['model_id'],
    load_dtype = '8-bit' if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    generation_config=GenerationConfig(**config['generation']['generation_config']),
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

# download dataset
data = get_DIALOCONAN_prepro(**config['generation']['custom_prompt'])
if config['data']['subset']:
    print('getting subset')
    # select only the first `subset_size` samples
    data = data[:config['data']['subset_size']]
dataset = get_dataset_CLM(
    data, 
    context_length = 512, 
    tokenizer = generator_manager.tokenizer
)

generator_manager.fine_tune(
    dataset = dataset, 
    val_set_per=.1,     # 10% of the dataset
    optimized = True,   # if torch.cuda.is_available() else False,
    lr = config['generation']['lr'],
    epochs = config['generation']['epochs'],
    initial_bs = config['generation']['initial_bs'],
    run_name = repo_id.split('/')[1],
)

# assuming debug if subset is active
if not config['data']['subset']:
    # save model to the hub
    generator_manager.push_to_hub(repo_id = repo_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: Loading binary /Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (no such file), '/Users/daniel/Documents/Work/RewardLM/.venv/

/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:33: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


Model loaded in fp32 (standard) mode


Map:   0%|          | 0/8312 [00:00<?, ? examples/s]

trainable params: 1179648 || all params 126378240 || trainable(%): 0.93
Trainer device: mps


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/351 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


: 

: 

### Load the fine tuned model from 🤗 Hub

In [5]:
from rewardlm.core.GenerativeModel import GenerativeModel
from transformers import GenerationConfig

new_generator = GenerativeModel(
    model_id=repo_id,
    load_from_peft=True,
)

print('Inference test: ')
inputs = new_generator.tokenizer("User: Woman should always be considere inferior?\nAssistant: ", return_tensors = 'pt')

new_generator.inference_fine_tuned(tokenized_batch=inputs, return_decoded=True)

Accelerator selected device: mps
Obtaining original model: MBZUAI/LaMini-GPT-124M
Inference test: 


/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


RuntimeError: MPS does not support cumsum op with int64 input

In [18]:
text = 'Explain how to solve the immigration problem'
inputs = new_generator.tokenizer(
    "User: {text}\nAssistant: ".format(text = text), 
    return_tensors = 'pt'
)
output = new_generator.inference_fine_tuned(tokenized_batch=inputs, return_decoded=True)
print(output)

User: Explain how to solve the immigration probelem
Assistant: 

To solve the immigration probelem, you need to follow these steps:

1. Identify the source of the probelem.
2. Identify the purpose of the probelem.
3. Determine the appropriate action to take.

Steps to solve the immigration probelem:

1. Identify the source of the probelem.
2. Identify the purpose of the probelem.
3. Determine the appropriate action to take.


# ⚖️ ToxicityMeter

In [4]:
import torch
from transformers import GenerationConfig
from rewardlm.core.GenerativeModel import GenerativeModel
from rewardlm.ToxicityMeter import ToxicityMeter
from rewardlm.data.data_utils import get_real_toxicity_prompts
from rewardlm.utils import load_config

config = load_config(name = 'debug_GPT-neo')

generator_manager = GenerativeModel(
    config['model_id'],
    load_dtype = config["generation"]["dtype"] if torch.cuda.is_available() else 'fp32',
    # force the use of CPU on Apple Silicon devices (mps not supported):
    generation_config=config['generation']['generation_config'],
    accelerator_kwargs = {
        'cpu': False if torch.cuda.is_available() else True,
    },
)

toxicity_meter = ToxicityMeter(generator_manager)
batchsize = 4
custom_prompt = (config['generation']['custom_prompt']['user_name'] + 
                 ' "{prompt}".\n' + 
                 config['generation']['custom_prompt']['bot_name'] + ' '
                )

df = get_real_toxicity_prompts()['text'].to_list()
toxicity_df = toxicity_meter.measure_toxicity(
    text_prompt = df if not config['data']['subset'] else df[:config['data']['subset_size']],
    custom_prompt = custom_prompt, 
    batch_size = batchsize,
    print_response=True,
)

Accelerator selected device: cpu
Model loaded in fp32 (standard) mode


In [ ]:
# save csv in tmp folder
fldr = './result analysis/tmp'
toxicity_df.to_csv(
    fldr + f'/measured_tox_instruct_{config["generation"]["model_id"].split("/")[-1]}_{config["generation"]["dtype"]}.csv'
)

# Just model testing here

In [1]:
model_id = 'EleutherAI/gpt-neo-125m'
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

text = ('### Question: Immigrants steal our jobs.. \n'
        '### Answer: Immigrants are often highly skilled workers wh [...] o contribute to our country by being employed in roles for which there are no national candidates.\n'
        '### Question: This is only an excuse to not hire native citizens!.\n'
        '### Answer: '
)
tokenized = tokenizer(text, return_tensors = 'pt')
output = model.generate(
    **tokenized,
    max_new_tokens = 256,
    min_new_tokens = 4,
    do_sample = False,
    # num_beams = 4,
    # early_stopping = False,
    # temperature = 0.7,
    # top_p = 0.7,
    # top_k = 50,
)

print(tokenizer.decode(output[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Question: Immigrants steal our jobs.. 
### Answer: Immigrants are often highly skilled workers wh [...] o contribute to our country by being employed in roles for which there are no national candidates.
### Question: This is only an excuse to not hire native citizens!.
### Answer: 

* * *

#                                                                                                                                                                                                                                                        


In [8]:
print(tokenizer.decode(output[0]))

Hello, my name is 

I'm a software developer and I'm looking for a good


# Interpretability tests

In [20]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

peft_model_id = "DanielSc4/gpt-neo-125m-RL-LoRA-8bit-test1"

config = LoraConfig.from_pretrained(peft_model_id)
original_pretrained = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
peft_model = get_peft_model(model = original_pretrained, peft_config=config)
print(peft_model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoForCausalLM(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(2048, 768)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0): GPTNeoBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): Linear(in_features=768, out_features=768, bias=False)
                (v_proj): Linear(
                  in_features=768, out_features=768, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=32, bias=False)

In [1]:
import inseq
inseq.list_feature_attribution_methods()

/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['attention',
 'discretized_integrated_gradients',
 'occlusion',
 'layer_deeplift',
 'gradient_shap',
 'layer_gradient_x_activation',
 'deeplift',
 'layer_integrated_gradients',
 'saliency',
 'input_x_gradient',
 'integrated_gradients',
 'lime']

In [85]:
peft_model_id = "DanielSc4/gpt-neo-125m-RL-LoRA-8bit-test1"

config = LoraConfig.from_pretrained(peft_model_id)
# TODO: come usare un modello peft (?)
seq_model = inseq.load_model(config.base_model_name_or_path, attribution_method='input_x_gradient')
seq_model

HuggingfaceDecoderOnlyModel(
  (model): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0.0, inplace=False)
      (h): ModuleList(
        (0): GPTNeoBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0.0, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(in_features=768, out_features=768, bias=False)
              (q_proj): Linear(in_features=768, out_features=768, bias=False)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_feat

In [94]:
out = seq_model.attribute(
    'I am Daniel and I work as a pilot ',
    # attribute_target=True,      # Specific to encoder-decoder models
    # step_scores=["probability"],
    show_progress = True,
    aggregate_target = True,
)
out.show()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Unused arguments during attribution: {'aggregate_target': True}
Attributing with input_x_gradient...: 100%|██████████| 20/20 [00:00<00:00, 13.88it/s]


,Ċ,in,Ġthe,ĠUnited,ĠStates,ĠAir,ĠForce,.,Ġ,Ċ
I,0.076,0.109,0.058,0.048,0.041,0.041,0.036,0.034,0.094,0.06
Ġam,0.07,0.074,0.048,0.043,0.028,0.03,0.021,0.025,0.05,0.035
ĠDaniel,0.059,0.053,0.086,0.084,0.046,0.042,0.019,0.023,0.059,0.032
Ġand,0.106,0.084,0.061,0.066,0.057,0.046,0.036,0.042,0.061,0.045
ĠI,0.045,0.039,0.038,0.026,0.021,0.022,0.011,0.013,0.026,0.025
Ġwork,0.074,0.071,0.073,0.063,0.031,0.047,0.02,0.023,0.032,0.028
Ġas,0.035,0.033,0.031,0.031,0.019,0.02,0.011,0.011,0.017,0.021
Ġa,0.053,0.047,0.028,0.037,0.032,0.025,0.016,0.018,0.033,0.038
Ġpilot,0.3,0.228,0.387,0.379,0.345,0.284,0.168,0.098,0.165,0.102
Ġ,0.181,0.096,0.023,0.025,0.025,0.017,0.021,0.016,0.115,0.071


In [103]:
out2 = seq_model.attribute(
    'I am Daniel and I work as a pilot ',
    # attribute_target=True,      # Specific to encoder-decoder models
    # step_scores=["probability"],
    show_progress = True,
    aggregate_target = False,
)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/daniel/Documents/Work/RewardLM/.venv/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Unused arguments during attribution: {'aggregate_target': False}
Attributing with input_x_gradient...: 100%|██████████| 20/20 [00:00<00:00, 13.95it/s]
